# 0. Requerimientos (carga, librerías)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. Carga data entrada

In [ ]:
data_entrada = pd.read_csv("../data/data_raw.csv", sep=",")
data_entrada.head(3)

# función: Cargar modelo (pickle)

In [ ]:
def cargar_modelo():
  with open('./models/prueba_model.pickle', 'rb') as file:
    modelo_pgh = pickle.load(file)
  return(modelo_pgh)

# función: procesamiento

In [ ]:
def procesamiento_categorias(df):
  temp_df = df.copy()

  #
  def grupo_calif(x):
    condiciones = [
        (x['NOR'] == 100),
        (x['PER'] >= 20),
        (x['DUD'] >= 20),
        (x['DEF'] >= 30),
        (x[['NOR', 'CPP', 'DEF', 'DUD', 'PER']].sum(axis=1) == 0)]
    valores = ['Normal', 'Perdida', 'Dudoso', 'Deficiente', 'Sin Calificación']
    grupos = ['1. Normal', '3. Deficiente/Dudoso/Perdida', '3. Deficiente/Dudoso/Perdida', '3. Deficiente/Dudoso/Perdida', '2. Sin Calificación/CPP']
    temp_df['calificacion'] = np.select(condiciones, valores, 'CPP')
    temp_df['calificacion_grupo'] = np.select(condiciones, grupos, '2. Sin Calificación/CPP')
  grupo_calif(temp_df)

  ##
  def grupo_datraso_sbs(x):
    if (0<=x) and (x<=15):
      return '1. 0 a 15'
    elif x>15:
      return '2. mas de 15'
    else: return 'ERROR'
  temp_df['max_datraso_sbs_grupo'] = temp_df.apply(lambda x: x[['MAXIMO_DIAS_ATRASO_DDIRECTASBS.1','MAXIMO_DIAS_ATRASO_DDIRECTASBS.2','MAXIMO_DIAS_ATRASO_DDIRECTASBS.3']].max(), axis=1)
  temp_df['max_datraso_sbs_grupo'] = temp_df['max_datraso_sbs_grupo'].apply(lambda x: grupo_datraso_sbs(x))

  ##
  def grupo_impagos(x):
    if x == 0:
      return '1. no tiene'
    else: return '2. tiene'
  temp_df['documentos_impagos_grupo'] = temp_df['Documentos Impagos'].apply(lambda x: grupo_impagos(x))



  #
  ##
  def grupo_dias_recurrente(x):
    if pd.isna(x):
      return '3. nuevo'
    elif int(x) == 0:
      return '1. 0'
    elif int(x) <= 60:
      return '2. 1 hasta 60'
    elif int(x) >60:
      return '4. 61 a más'
    else: return 'ERROR'
  temp_df['datraso_p2p_grupo'] = temp_df['Dias de atraso de PAGO'].apply(lambda x: grupo_dias_recurrente(x))



  #
  ##
  def grupo_ocupacion(x):
    if x['sector_detalle'] == 'otros servicios: arrendamiento':
      return '3. arrendamiento'
    elif x['sector'] == 'dependiente':
      return '2. dependiente'
    else: return '1. independiente'
  temp_df['ocupacion_grupo'] = temp_df[['sector','sector_detalle']].apply(grupo_ocupacion, axis=1)

  ##
  def grupo_al_menos_uno_vive(x):
    if x['al_menos_uno_de_los_solicitantes_vive_en_la_garantia_'] == 'VERDADERO':
      return '2. SÍ'
    else: return '1. NO'
  temp_df['al_menos_uno_vive_en_la_garantia_grupo'] = temp_df.apply(grupo_al_menos_uno_vive, axis=1)

  ##
  def grupo_motivo_principal(x):
    if (x=='Expansión del Negocio') or (x=='Lanzar nuevo negocio') or (x=='Capital de trabajo'):
      return '2. Capital de trabajo'
    elif (x=='Compra de vehículo') or (x=='Otros motivos personales') or (x=='Inversión para terceros')  or (x=='No hay información'):
      return '4. Otros motivos personales'
    elif (x=='Construir o remodelar') or (x=='Construir o remodelar local comercial para uso propio') or (x=='Construir o remodelar local comercial para alquiler'):
      return '1. Construir o remodelar'
    elif x=='Compra de deuda':
      return '3. Compra de deuda'
    else: return '4. Otros motivos personales'
  temp_df['motivo_principal_grupo'] = temp_df['motivo_principal_del_prestamo'].apply(lambda x: grupo_motivo_principal(x))



  #
  def grupo_distritos(distrito):
    lista_distritos1 = [
        'ancon', 'barranca', 'bellavista', 'cerro colorado', 'cieneguilla', 'lince',
        'lurin', 'paucarpata', 'punta hermosa', 'santa anita', 'yanahuara']
    lista_distritos2 = [
        'ate', 'breña', 'callao', 'carabayllo', 'chorrillos', 'comas', 'la victoria',
        'los olivos', 'magdalena del mar', 'miraflores', 'puente piedra', 'san bartolo',
        'san juan de lurigancho', 'san juan de miraflores', 'san martin de porres',
        'san miguel', 'santiago de surco', 'villa el salvador', 'villa maria del triunfo']
    lista_distritos3 = [
        'arequipa', 'asia', 'barranco', 'castilla', 'catacaos', 'cayma', 'cercado de lima', 'cerro azul',
        'chaclacayo', 'chilca', 'chincha', 'chincha alta', 'el agustino', 'el porvenir', 'el tambo',
        'florencia de mora', 'huancayo', 'ica', 'independencia', 'jesus maria','jose luis bustamante y rivero',
        'junin', 'la esperanza', 'la molina', 'la perla', 'lurigancho-chosica', 'mariano melgar',
        'pachacamac', 'pucusana', 'pueblo libre', 'rimac', 'san borja', 'san isidro', 'san luis',
        'santa rosa', 'surquillo', 'trujillo', 'ventanilla', 'victor larco herrera']

    if distrito in lista_distritos1:
      return '1. grupo 1'
    elif distrito in lista_distritos2:
        return '2. grupo 2'
    elif distrito in lista_distritos3:
        return '3. grupo 3'
    else: return '3. grupo 3'
  temp_df['distritos_grupo'] = temp_df['Distrito de Garantia'].apply(lambda x: grupo_distritos(x))



  # Salida
  return(temp_df[[
             'calificacion_grupo',
             'max_datraso_sbs_grupo', 'documentos_impagos_grupo',
             'datraso_p2p_grupo', 'ocupacion_grupo',
             'al_menos_uno_vive_en_la_garantia_grupo', 'motivo_principal_grupo',
             'distritos_grupo']])

In [ ]:
def procesamiento_modelo(df):
  temp_df = procesamiento_categorias(df.copy())
  #temp_df = df.copy()

  # Recodificación
  mapeo_ordinal = {
    '1. no tiene': 1,
    '2. tiene': 2,

    '1. 0 a 15': 1,
    '2. mas de 15': 2,

    '1. NO': 1,
    '2. SÍ': 2,

    '1. independiente': 1,
    '2. dependiente': 2,
    '3. arrendamiento': 3,

    '1. grupo 1': 1,
    '2. grupo 2': 2,
    '3. grupo 3': 3,

    '1. Normal': 1,
    '2. Sin Calificación/CPP': 2,
    '3. Deficiente/Dudoso/Perdida': 3,

    '1. 0': 1,
    '2. 1 hasta 60': 2,
    '3. nuevo': 3,
    '4. 61 a más': 4,

    '1. Construir o remodelar': 1,
    '2. Capital de trabajo': 2,
    '3. Compra de deuda': 3,
    '4. Otros motivos personales': 4,

    'a. no tiene': 1,
    'b. tiene': 2,

    'a. 0 a 15': 1,
    'b. mas de 15': 2,

    'a. NO': 1,
    'b. SÍ': 2,

    'a. independiente': 1,
    'b. dependiente': 2,
    'c. arrendamiento': 3,

    'a. grupo 1': 1,
    'b. grupo 2': 2,
    'c. grupo 3': 3,

    'a. Normal': 1,
    'b. Sin Calificación/CPP': 2,
    'c. Deficiente/Dudoso/Perdida': 3,

    'a. 0': 1,
    'b. 1 hasta 60': 2,
    'c. nuevo': 3,
    'd. 61 a más': 4,

    'a. Construir o remodelar': 1,
    'b. Capital de trabajo': 2,
    'c. Compra de deuda': 3,
    'd. Otros motivos personales': 4}

  for column in ['calificacion_grupo', 'max_datraso_sbs_grupo', 'documentos_impagos_grupo', 'datraso_p2p_grupo', 'ocupacion_grupo', 'al_menos_uno_vive_en_la_garantia_grupo', 'motivo_principal_grupo', 'distritos_grupo']:
    temp_df.loc[:, column] = temp_df[column].replace(mapeo_ordinal)
    temp_df[column] = temp_df[column].astype(int)



  # Salida
  return(temp_df)


# función: Score modelo

In [ ]:
def score_modelo(df):
  columnas_orden_modelo = ['const', 'documentos_impagos_grupo', 'max_datraso_sbs_grupo', 'al_menos_uno_vive_en_la_garantia_grupo',
                           'ocupacion_grupo', 'distritos_grupo', 'calificacion_grupo', 'datraso_p2p_grupo', 'motivo_principal_grupo']

  temp_df = df.copy()
  temp_df['const'] = 1
  temp_df = temp_df[columnas_orden_modelo]
  # Cálculo de 'prob' y 'score'
  df_score = pd.DataFrame({'fecha_proceso':pd.to_datetime('today').date(), 'score':modelo_pgh.predict(temp_df)})
  df_score['score'] = 1000 - 1000*df_score['score']
  df_score['score'] = df_score['score'].apply(lambda x: int(np.floor(x)))

  # Tramos de puntajes
  tramos = [
      (df_score['score'] <= 670),
      (df_score['score'].between(671, 755)),
      (df_score['score'].between(756, 860)),
      (df_score['score'].between(861, 915)),
      (df_score['score'].between(916, 945)),
      (df_score['score'] > 945)]
  niveles = ['E', 'D', 'C', 'B', 'A', 'A+']

  # 'Nivel_de_riesgo'
  df_score['Nivel_de_riesgo'] = pd.Categorical(np.select(tramos, niveles, default='Sin clasificación'))

  return(df_score)
